In [84]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import os
import glob

import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
#import tensorflow_decision_forests as tfdf

print(tf.__version__)

# Make NumPy printouts easier to read.
np.set_printoptions(precision=3, suppress=True)

2.8.0


### Club all the traces together into a single dataframe

In [85]:
path = './traces/'# use your path
#all_files = glob.glob(path + "/*.h5")
all_files = glob.glob(path + "/dpotrf_T*.prof.h5")

li = []

for filename in all_files:
    trace =  pd.HDFStore(filename)
    data = trace.get('/events')
    trace.close()
    
    li.append(data)

#df = pd.concat(li, axis=0, ignore_index=True)


In [86]:
df_class_name = {0: 'dpotrf', 1: 'dtrsm', 2: 'dsyrk', 3: 'dgemm'}
print(len(li))

8


# Clean the data

In [87]:
#remove unwanted columns
for i in range(0, len(li)):
    li[i] = li[i][['begin', 'end', 'taskpool_id', 'task_class_id','chore_id', 'nb_data_items', 'total_data_size', 'priority']].copy()
    li[i]['exec_time'] = (li[i]['end'] - li[i]['begin']) * .001 #0.001 nano seconds to micro seconds
    li[i] = li[i].astype(float)

#remove all other classes other than dpotrf, dgemm, trsm and syrk
for i in range(0, len(li)):
    filter = ( (li[i]['task_class_id'] >= 0 ) & ( li[i]['task_class_id'] <= 3 ) & ( li[i]['priority'] >= 0 ) )
    li[i] = li[i][filter]
    #df_refined.head()

### remove outliers

In [88]:
li_new = []
for i in range(0, len(li)):
    df_li = li[i]
    for j in df_li['task_class_id'].unique():
        filter = (li[i]['task_class_id'] == j )
        df_li_class = df_li[filter]
        Q1 = df_li_class['exec_time'].quantile(0.25)
        Q3 = df_li_class['exec_time'].quantile(0.75)
        IQR = Q3 - Q1    #IQR is interquartile range.
        filter = (df_li_class['exec_time'] >= Q1 - 1.5 * IQR) & (df_li_class['exec_time'] <= Q3 + 1.5 *IQR)
        df_li_class = df_li_class.loc[filter]
        df_li_class = df_li_class.astype(float)
        li_new.append(df_li_class)
                  
print(len(li_new)) 

32


## integrate likelihood of the data being in cash.
we assume that all the task whose execution time is in the first quartile could have had all its data in the cache. While, the rest of the tasks would have resulted in a cache flush. While predicting we assume that the that the data of the task is not in the cache 

In [89]:
for i in range(0, len(li_new)):
    Q1 = li_new[i]['exec_time'].quantile(0.25)
    Q3 = li_new[i]['exec_time'].quantile(0.75)
    IQR = Q3 - Q1    #IQR is interquartile range.
    li_new[i]['cache_likelihood'] = np.where(li_new[i]['exec_time'] < Q1, 1, 0)

### New DataFrame

In [90]:
df_new = pd.concat(li_new, axis=0, ignore_index=True)
df_new.head()

,begin,end,taskpool_id,task_class_id,chore_id,nb_data_items,total_data_size,priority,exec_time,cache_likelihood
0,199680.0,6506370.0,2.0,1.0,0.0,2.0,5120000.0,0.0,6306.690,0
1,6530386.0,15152069.0,2.0,1.0,0.0,2.0,5120000.0,0.0,8621.683,0
2,15171375.0,21961048.0,2.0,1.0,0.0,2.0,5120000.0,0.0,6789.673,0
3,21978558.0,28177033.0,2.0,1.0,0.0,2.0,5120000.0,0.0,6198.475,0
4,28186947.0,34587327.0,2.0,1.0,0.0,2.0,5120000.0,0.0,6400.380,0


## change task_class column to catogrical data

In [91]:
#df_refined_new = pd.get_dummies(df_refined_new, prefix='task_pool_', columns=['taskpool_id'], prefix_sep='')
df_new = pd.get_dummies(df_new, prefix='task_class_', columns=['task_class_id'], prefix_sep='')

In [92]:
df_new.head()

,begin,end,taskpool_id,chore_id,nb_data_items,total_data_size,priority,exec_time,cache_likelihood,task_class_0.0,task_class_1.0,task_class_2.0,task_class_3.0
0,199680.0,6506370.0,2.0,0.0,2.0,5120000.0,0.0,6306.690,0,0,1,0,0
1,6530386.0,15152069.0,2.0,0.0,2.0,5120000.0,0.0,8621.683,0,0,1,0,0
2,15171375.0,21961048.0,2.0,0.0,2.0,5120000.0,0.0,6789.673,0,0,1,0,0
3,21978558.0,28177033.0,2.0,0.0,2.0,5120000.0,0.0,6198.475,0,0,1,0,0
4,28186947.0,34587327.0,2.0,0.0,2.0,5120000.0,0.0,6400.380,0,0,1,0,0


### Divide the data to training and test sets

In [95]:
train_dataset = df_new.sample(frac=0.8, random_state=0)
test_dataset = df_new.drop(train_dataset.index)
print(train_dataset.shape)
print(test_dataset.shape)

(860154, 13)
(215038, 13)


## Divide the data into trainig features and labels

In [96]:
train_features = train_dataset.copy()
test_features = test_dataset.copy()

train_labels = train_features.pop('exec_time')
test_labels = test_features.pop('exec_time')

In [97]:
#train_features.describe().transpose()

# -------------------Build the multi variate full model------------------------------

## select the required features from the list

In [98]:

train_features = train_features.drop(['chore_id', 'begin', 'end', 'taskpool_id'], axis = 1)
test_features = test_features.drop(['chore_id', 'begin', 'end', 'taskpool_id'], axis = 1)



In [99]:
train_features.head()

,nb_data_items,total_data_size,priority,cache_likelihood,task_class_0.0,task_class_1.0,task_class_2.0,task_class_3.0
267930,2.0,320000.0,0.0,1,0,1,0,0
373649,3.0,960000.0,1268.0,0,0,0,0,1
871026,3.0,240000.0,10856.0,0,0,0,0,1
942871,3.0,240000.0,248954.0,0,0,0,0,1
414417,3.0,960000.0,146104.0,0,0,0,0,1


In [100]:
normalizer = tf.keras.layers.Normalization(axis=-1)
#normalizer = layers.Normalization(input_shape=[1,], axis=None)
normalizer.adapt(np.array(train_features))

In [101]:
LR_model = tf.keras.Sequential([
    normalizer,
    layers.Dense(units=1)
])

In [102]:
LR_model.compile(
    optimizer=tf.optimizers.Adam(learning_rate=0.1),
    loss='mse', metrics=[tf.keras.metrics.Accuracy()])
print(LR_model.metrics_names)

[]


In [103]:
checkpoint_path = "LR_model_all_class.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)
lr_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

## Train the model

In [104]:
history = LR_model.fit(
    train_features,
    train_labels,
    epochs=1,
    # Suppress logging.
    verbose=0,
    # Calculate validation results on 20% of the training data.
    validation_split = 0.2,
    callbacks=[lr_callback])


Epoch 1: saving model to LR_model_all_class.ckpt


In [66]:
LR_model.save('./LR_model_all_class')

INFO:tensorflow:Assets written to: ./LR_model_all_class/assets


In [105]:
history.history

{'loss': [17170252.0],
 'accuracy': [0.0],
 'val_loss': [13176643.0],
 'val_accuracy': [0.0]}

In [106]:
LR_model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 normalization_2 (Normalizat  (None, 8)                17        
 ion)                                                            
                                                                 
 dense_5 (Dense)             (None, 1)                 9         
                                                                 
Total params: 26
Trainable params: 9
Non-trainable params: 17
_________________________________________________________________


## Train using DNN

In [107]:
def build_and_compile_model(norm):
  model = keras.Sequential([
      norm,
      layers.Dense(64, activation='relu'),
      layers.Dense(64, activation='relu'),
      layers.Dense(1)
  ])

  model.compile(loss='mean_absolute_error',
                optimizer=tf.keras.optimizers.Adam(0.001))
  return model

In [108]:
dnn_model = build_and_compile_model(normalizer)
dnn_model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 normalization_2 (Normalizat  (None, 8)                17        
 ion)                                                            
                                                                 
 dense_6 (Dense)             (None, 64)                576       
                                                                 
 dense_7 (Dense)             (None, 64)                4160      
                                                                 
 dense_8 (Dense)             (None, 1)                 65        
                                                                 
Total params: 4,818
Trainable params: 4,801
Non-trainable params: 17
_________________________________________________________________


In [109]:
checkpoint_path = "DNN_model_all_class.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)
dnn_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

In [ ]:
history = dnn_model.fit(
    train_features,
    train_labels,
    validation_split=0.2,
    verbose=0, epochs=100,
    callbacks=[dnn_callback])


Epoch 1: saving model to DNN_model_all_class.ckpt

Epoch 2: saving model to DNN_model_all_class.ckpt

Epoch 3: saving model to DNN_model_all_class.ckpt

Epoch 4: saving model to DNN_model_all_class.ckpt

Epoch 5: saving model to DNN_model_all_class.ckpt

Epoch 6: saving model to DNN_model_all_class.ckpt

Epoch 7: saving model to DNN_model_all_class.ckpt

Epoch 8: saving model to DNN_model_all_class.ckpt

Epoch 9: saving model to DNN_model_all_class.ckpt

Epoch 10: saving model to DNN_model_all_class.ckpt

Epoch 11: saving model to DNN_model_all_class.ckpt

Epoch 12: saving model to DNN_model_all_class.ckpt

Epoch 13: saving model to DNN_model_all_class.ckpt

Epoch 14: saving model to DNN_model_all_class.ckpt

Epoch 15: saving model to DNN_model_all_class.ckpt

Epoch 16: saving model to DNN_model_all_class.ckpt

Epoch 17: saving model to DNN_model_all_class.ckpt

Epoch 18: saving model to DNN_model_all_class.ckpt

Epoch 19: saving model to DNN_model_all_class.ckpt

Epoch 20: saving mod

In [111]:
dnn_model.save('DNN_all_class')


INFO:tensorflow:Assets written to: DNN_all_class/assets


In [112]:
history.history

{'loss': [254.07647705078125], 'val_loss': [194.34872436523438]}